In [1]:
import os
from os.path import join, pardir, curdir
import glob
import numpy as np
import pandas as pd
import scipy as sp
from tqdm import tqdm
from matplotlib import pyplot as plt
from joblib import load, dump
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, precision_score, classification_report

# Set current directory to "src"
os.chdir(join(os.getcwd(), os.pardir, "src"))
print(f"Current working directory: {os.getcwd()}")

from feature_extractor import *
from process import *

Current working directory: /home/fa926284/Documents/Parkinson_FOG_Detection/src


In [2]:
DATASETS = ['tdcsfog', 'defog']
DATA_PATH = join(pardir, 'data')
PROCESSED_DATA_PATH = join(pardir, "data", "processed")
MODEL_PATH = join(pardir, "models")
RANDOM_SEED = 42

TEST_DIRS = []

for dataset in DATASETS:
    TEST_DIRS.extend(glob.glob(join(DATA_PATH, "test", dataset, "*.csv"))[:])


In [14]:
X_test0 = load(join(PROCESSED_DATA_PATH, "X_test0.joblib"))
print(X_test0.shape)
X_test1 = load(join(PROCESSED_DATA_PATH, "X_test1.joblib"))
print(X_test1.shape)

model = load(join(MODEL_PATH, 'model_rf.sav'))
y_pred0 = model.predict(X_test0)
# y_pred0 = RunningMedian(y_pred0, 50)
y_pred1 = model.predict(X_test1)
# y_pred1 = RunningMedian(y_pred1, 50)


(4583, 66)
(281589, 66)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished


In [54]:
test_df0 = pd.read_csv(TEST_DIRS[0])
test_df1 = pd.read_csv(TEST_DIRS[1])

y_pred0_final = np.full(shape=test_df0.shape[0],
                        fill_value=int(np.median(y_pred0[-99:])))
y_pred1_final = np.full(shape=test_df1.shape[0],
                        fill_value=int(np.median(y_pred1[-99:])))

y_pred0_final[:y_pred0.shape[0]] = y_pred0
y_pred1_final[:y_pred1.shape[0]] = y_pred1

y_pred0_final = label_binarize(y_pred0_final, classes=[0, 1, 2, 3])[:, 1:]
y_pred1_final = label_binarize(y_pred1_final, classes=[0, 1, 2, 3])[:, 1:]

print(y_pred0_final.shape, y_pred1_final.shape)


(4682, 3) (281688, 3)


In [60]:
sub0 = pd.DataFrame(columns=["ID", "StartHesitation", "Turn", "Walking"])
sub0['ID'] = os.path.basename(TEST_DIRS[0]).split('.')[0] + "_" + test_df0['Time'].astype(str)
sub0['StartHesitation'] = y_pred0_final[:, 0]
sub0['Turn'] = y_pred0_final[:, 1]
sub0['Walking'] = y_pred0_final[:, 2]

sub0['StartHesitation'].value_counts(), sub0['Turn'].value_counts(), sub0['Walking'].value_counts()

(StartHesitation
 0    4680
 1       2
 Name: count, dtype: int64,
 Turn
 0    3896
 1     786
 Name: count, dtype: int64,
 Walking
 0    4682
 Name: count, dtype: int64)

In [61]:
sub1 = pd.DataFrame(columns=["ID", "StartHesitation", "Turn", "Walking"])
sub1['ID'] = os.path.basename(TEST_DIRS[1]).split('.')[0] + "_" + test_df1['Time'].astype(str)
sub1['StartHesitation'] = y_pred1_final[:, 0]
sub1['Turn'] = y_pred1_final[:, 1]
sub1['Walking'] = y_pred1_final[:, 2]

sub1['StartHesitation'].value_counts(), sub1['Turn'].value_counts(), sub1['Walking'].value_counts()

(StartHesitation
 0    281688
 Name: count, dtype: int64,
 Turn
 0    281153
 1       535
 Name: count, dtype: int64,
 Walking
 0    281686
 1         2
 Name: count, dtype: int64)

In [68]:
submission = pd.concat([sub0, sub1], axis=0)

submission.to_csv(join(DATA_PATH, "submission.csv"))
submission

,ID,StartHesitation,Turn,Walking
0,003f117e14_0,0,0,0
1,003f117e14_1,0,0,0
2,003f117e14_2,0,0,0
3,003f117e14_3,0,0,0
4,003f117e14_4,0,0,0
...,...,...,...,...
281683,02ab235146_281683,0,0,0
281684,02ab235146_281684,0,0,0
281685,02ab235146_281685,0,0,0
281686,02ab235146_281686,0,0,0
